In [1]:
%load_ext autoreload
%autoreload
%matplotlib inline

import os
import sys
parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)

from newscrawler import NewsCrawler
from IPython.display import display
from IPython.core.debugger import set_trace

# 0. 초기화

In [8]:
crawler = NewsCrawler(storage='bigquery')

checking bigquery storage... done (28.56 seconds)
we have total 371169 articles (328816 downloaded, 42353 trashed)


# 1. URL을 모은다

In [18]:
# 제거고려: nextbigfuture, nbcnews, joongang, chosun

In [4]:
collect_result = crawler.collect(); display(*collect_result)

100.00% (113.21 seconds): Collecting URLs... yahoo               

100.00% (0.12 seconds): Selecting URLs... cryptonews          

,collected,collected_unique,selected
all,26112,26039,348
reuters,1072,1072,43
investing.com,908,908,38
yahoo,361,361,37
dailymail,2740,2740,27
...,...,...,...
techcrunch,20,20,0
morningstar,108,108,0
bbc,277,277,0
nbcnews,178,177,0


,url,pubs,actual_pub
0,https://politico.com/news/2019/11/02/trump-hos...,"politico, realclearpolitics",politico
1,https://politico.com/magazine/story/2019/10/30...,"politico, realclearpolitics",politico
2,https://politico.com/news/2019/11/02/beto-orou...,"politico, realclearpolitics",politico
3,https://politico.com/magazine/story/2019/10/23...,"politico, realclearpolitics",politico
4,https://politico.com/magazine/story/2019/11/01...,"politico, realclearpolitics",politico
...,...,...,...
68,https://nbcnews.com/politics/trump-impeachment...,"msnbc, nbcnews",nbcnews
69,https://nbcnews.com/news/us-news/florida-teen-...,"msnbc, nbcnews",nbcnews
70,https://nbcnews.com/health/health-care/former-...,"msnbc, nbcnews",nbcnews
71,https://nbcnews.com/news/world/massive-ship-st...,"msnbc, nbcnews",nbcnews


# 2. 해당 URL의 뉴스를 크롤링한다

In [5]:
crawl_result = crawler.crawl(); crawl_result

100.00% (212.79 seconds): Crawling... yahoo               



collect                                crawl              
              collected collected_unique selected downloaded trashed total
all               26112            26039      348        233     115   348
reuters            1072             1072       43         27      16    43
investing.com       908              908       38         18      20    38
yahoo               361              361       37         33       4    37
dailymail          2740             2740       27         24       3    27
...                 ...              ...      ...        ...     ...   ...
techcrunch           20               20        0          0       0     0
morningstar         108              108        0          0       0     0
bbc                 277              277        0          0       0     0
nbcnews             178              177        0          0       0     0
cryptonews          328              328        0          0       0     0

[74 rows x 6 columns]

# 3. 뉴스를 저장한다

In [6]:
crawler.record()

Recording to bigquery storage... done (17.77 seconds)


# 4. 결과보고

In [7]:
crawler.report()

In [66]:
from pathlib import Path
downloaded = [p.stem for p in Path('newsdata/downloaded').glob('**/*.json')]
trashed = [p.stem for p in Path('newsdata/trashed').glob('**/*.json')]

In [65]:
for path in list(Path('newsdata/downloaded').glob('**/*.json'))[:10000]:
    path.unlink()

In [53]:
from pathlib import Path
import json
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.header import Header

In [54]:
acc_fname = 'smtp_gmail.json'
acc = json.loads(Path(acc_fname).read_text()); acc

{'host': 'smtp.gmail.com',
 'from': 'gem763@gmail.com',
 'to': ['gem763@gmail.com', 'sejin.kang@hanwha.com'],
 'pw': 'kkangse1'}

In [55]:
msg = MIMEMultipart('alternative')
msg['Subject'] = Header('NEWS CRAWLING REPORT', 'utf-8')
msg['From'] = acc['from']
msg['To'] = ', '.join(acc['to'])
msg.attach(MIMEText(crawler.crawl_summary.to_html(), 'html'))

with smtplib.SMTP_SSL(acc['host']) as smtp:
    smtp.login(acc['from'], acc['pw'])
    smtp.send_message(msg)

In [1]:
import pandas as pd

In [13]:
str(pd.Timestamp.utcnow())[:19]

'2019-11-03 05:39:06'

In [15]:
import json
type(json.load(open('smtp_gmail.json')))

dict